In [ ]:
# Start writing code here...
# # store list of quantitative variables
# quants = ['APOGEE', 'PERIGEE', 'INCLINATION', 'PERIOD']
# # initialize list to store boolean series of missing / non-missing data
# missing = []
# for i in quants:
#     nones = all_objects[i].isnull()
#     print(sum(nones))
#     missing.append(nones)

# # Noting that all counts were the same, check if all null values go together
# either_or = all_objects[all_objects['APOGEE'].isnull() & all_objects['PERIGEE'].notna()]
# print("The number of observations with missing APOGEE value but not missing PERIGEE is", either_or.shape[0])

# # Noting that all missing values are in the same observations, grab these observations
# gone = all_objects[all_objects['APOGEE'].isnull()]
# print("The subset of all observations with missing quantitative values is:")
# display(gone.head(10))

# # Check all unique comments to identify why missing values
# comments = gone.COMMENT.unique()
# print("The various types of observations that yield missing values are:")
# print(comments)

# # store indices of incomplete observations, taken from gone dataframe
# #incomplete_indices = gone.index

# # drop these incomplete rows from in_orbit dataframe
# #all_objects = all_objects.drop(index = incomplete_indices)
# all_objects = all_objects.dropna(subset=['APOGEE'])


In [ ]:
# Check key non-quantitative variables for missing values
# c = sum(all_objects['COUNTRY'].isnull())
# l = sum(all_objects['LAUNCH'].isnull())

# print(f"There are {c} missing countries in the COUNTRY feature.")
# print(f"There are {l} missing launch dates in the LAUNCH feature.")

To ensure that our later analyses occur without problems, we assess missing values in our dataset.
The most important features to evaluate for missing values are the numerical variables, launch date, and "country,"
as these are the features that are promising predictors for our models. We find that there are no missing values 
in the launch data and country features. On the other hand, there are 901 missing values in each numerical 
feature (which is not incredibly high, given that there are over 40,000 total observatons). We analyze these 
and find that the missing values are not random: they all come from the same 901 observations.
When we subset these out from the full dataframe, we find that they correspond to just a few _types_ of observations
(as described by the COMMENTS). For example, some of them correspond to space objects that are in solarcentric orbit.
This means they do not orbit the earth, but rather the sun. Other categories include lunarcentric objects, 
objects in Mars orbit, and objects that have made impact with Venus. Therefore, the values for apogee, perigee, 
inclination, and period are missing because they cannot be calculated -- they do not apply for objects not 
currently in Earth orbit. We thus decide to not impute for our missing data. Rather, we drop these rows as they 
do not factor into our analysis of space debris _surrounding Earth_.

We evaluated the distributions of our numerical predictors, running .describe() in order to attain mean,
standard deviation, and quartiles. We note immediately that our data is wildly skewed. The standard deviation
of the APOGEE datapoints is about 14,000; PERIGEE's is about 7,300. As another indicator: 
While the third quartile for APOGEE is 1011, the max value is almost 700,000. INCLINATION shows far less spread,
although PERIOD has a similarly wide gap between the top of its interquartile range and its max value: about 95,400.
Thus, before plotting our distributions and moving ahead with our analysis, we determine the outlier values. Employing
the common outlier quantification (Q3 + (1.5* IQR), Q1 - (1.5*IQR)), we determine that there are over 5,000 "outlier"
values in both the APOGEE and PERIOD features, and another 2,925 in PERIGEE (none in INCLINATION, as expected). Of
course, we do expect there to be high overlap in the outliers from each feature (ie a satellite with high APOGEE 
should also have high PERIGEE, unless it has a very elliptical orbit). When we plot the numerical datapoints sans
outliers, we find the above violin plot distributions. APOGEE and PERIGEE are still right skewed; PERIOD and INCLINATION
have much narrower spreads. The bulk of APOGEE's points are between about 0 and 500, while PERIGEE's are between
about 0 and 250. Even though we don't drop the rows with outliers (as we still need data on those launches for our
other analyses), this evaluation of outliers is important for our graphing below (in which we restrict the range of values 
included) and for any further work we do with the numerical features (in which we might drop outliers).


In [ ]:
### 3D SPATIAL DISTRIBUTION OF OBJECTS IN LEO

# create a copy of the in_orbit dataframe 
df = in_orbit.copy()

# subset dataframe to only those points that are not outliers
# determined via the graphs in the cell above
# easier for viewing, and removes objects that may be at 
# Lagrange points (not near Earth), for example
df = df[df['APOGEE'] < 2500]
df = df[df['PERIGEE'] < 1700]

# select a random set of 2000 datapoints for easier viewing
df = df.loc[np.random.choice(df.index, 2000, replace=False)]
a = df['APOGEE'].to_numpy()
b = df['PERIGEE'].to_numpy()

# randomly generate a number between 0 and 2pi
# this is essentially the longitude of the object's
# location around Earth
omega = 2*np.pi * np.random.random(len(a))

# using the formula for an ellipse in polar coordinates,
# find the position of the space object such that it falls on
# the object's elliptical flight path (given by APOGEE and PERIGEE)
# and is at the randomly sampled longitude identified above
b_sq = np.multiply(np.square(b), np.square(np.cos(omega)))
a_sq = np.multiply(np.square(a), np.square(np.sin(omega)))

r = np.divide(np.multiply(a,b), np.sqrt(a_sq + b_sq))

# store list of positions and longitudes in new features
df['R'] = r
df['OMEGA'] = omega

# SPACE OBJECTS LAUNCHED BY US
df_y = df[df['COUNTRY'] == "US"]

# convert from polar to Cartesian in order to plot
x = np.multiply(np.multiply(df_y['R'], np.cos(df_y['INCLINATION'])), np.cos(df_y['OMEGA']))
y = np.multiply(np.multiply(df_y['R'], np.cos(df_y['INCLINATION'])), np.sin(df_y['OMEGA']))
z = np.multiply(df_y['R'],  np.sin(df_y['INCLINATION']))

# same thing, except for OBJECTS FROM NON-US COUNTRIES
df_non = df[df['COUNTRY'] != "US"]

x2 = np.multiply(np.multiply(df_non['R'], np.cos(df_non['INCLINATION'])), np.cos(df_non['OMEGA']))
y2 = np.multiply(np.multiply(df_non['R'], np.cos(df_non['INCLINATION'])), np.sin(df_non['OMEGA']))
z2 = np.multiply(df_non['R'],  np.sin(df_non['INCLINATION']))

# plot with plotly 
plotly.offline.init_notebook_mode()

# create trace for "US object" datapoints
# same thing below for non-US and for the Earth
trace = go.Scatter3d(
    x=x, 
    y=y,  
    z=z, 
    name = "US Objects",
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
    }
)

trace1 = go.Scatter3d(
    x=x2,  
    y=y2,  
    z=z2,  
    name = "Non-US Objects",
    mode='markers',
    marker={
        'size': 5,
        'opacity': 0.8,
        'color': 'green',
    }
)

trace2 = go.Scatter3d(
    x=[0],  
    y=[0],  
    z=[0],  
    name = 'Earth',
    mode='markers',
    marker={
        'size': 20,
        'opacity': 0.8,
        'color': 'brown',
    }
)

fig = make_subplots(specs=[[{"type": 'scene'}]])
fig.add_trace(trace)
fig.add_trace(trace1)
fig.add_trace(trace2)
fig['layout'].update(height = 600, width = 800, title = 'Debris Spatial Distribution',xaxis=dict(
      tickangle=-90
    ))
fig.show()


The 3D graph above depicts the spatial distribution of functional space objects currently in orbit around the Earth. 
The points are color coded such that blue points represent US objects and green points represent non-US satellites.
The graph was constructed via Plotly, which we learned in order to implement a clear, interactive 3D plot.
A random sample of 2,000 datapoints (space objects) is selected, given that the total 20,000+ observations would 
overwhelm the plot. Moreover, objects are plotted at a random location along their elliptical flight paths -- all
we are given in the dataset is inclination and distance from Earth ("r" in polar coordinates). Thus, randomly sampling
from along the flight path returns the location of the object. This graph tells us a few important things: first,
we see that while the preponderence of space objects are not from the US, the US has launched most of the furthest
objects from the center of te Earth. Indeed, non-US objects are clumped above the Poles, while the US' are either more 
dispersed or located in a ring above what may be a Tropic. Secondly, space is _very_ dense. Recall that this is only
a fraction of the total data. Assuming the rest are distributed similarly (which we can assume so long as our sample
is representative), space around the Poles, particularly, is in short supply (of course, our plot does not do a good
job of quantifying just how much empty space there is, but when we also consider that decayed objects are not included
on the chart, we understand that this space is indeed quite dense). Third, this type of plot may be useful in
our later analyses -- we can change the color coding to signify launch date, per se, rather than country of origin. 
In this case, we would be able to assess spatially, in 3D, changes in distribution, object density, and object distance
over time.

In [ ]:
# FUNDING AND SPACE ACTIVITY
# read in dataset with NASA funding
funding = pd.DataFrame(pd.read_csv("data/nasa_budget.csv"))[['Fiscal Year', 'Request', 'Actual']]
funding = funding[funding['Fiscal Year'] < 2020]

funding['Actual'] = pd.to_numeric(funding['Actual'])

funding_all = funding.merge(all_objects, left_on = 'Fiscal Year', right_on = 'LAUNCH_YEAR')

# extract and store number of launches overall and for US
number_of_launches = funding_all.groupby('LAUNCH_YEAR').count()['Fiscal Year']
number_of_launches_us = funding_all[funding_all['COUNTRY'] == 'US'].groupby('LAUNCH_YEAR').count()['Fiscal Year']
number_of_launches = funding_all.groupby("OBJECT_NAME").mean()["LAUNCH_YEAR"].astype('int').value_counts().sort_index()

# merge funding and US launches dataframe subsets
us_vs_funding = funding.merge(pd.DataFrame(number_of_launches_us), left_on = "Fiscal Year", right_index = True)


fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(28,14))
ax[0].plot(funding[funding['Fiscal Year'] < 2020]['Fiscal Year'], funding['Actual'], c = "blue")
ax[0].set_title("Funding vs. Year", fontsize = '24')
ax[0].set_xlabel("Year")
ax[0].set_ylabel("Funding")
ax[1].plot(funding[funding['Fiscal Year'] < 2021]['Fiscal Year'], number_of_launches_us, c = "blue")
ax[1].set_title("Number of US Launches vs. Year", fontsize = '24')
ax[1].set_xlabel("Year")
ax[1].set_ylabel("Number of US Launches")
ax[2].scatter( x= us_vs_funding['Actual'],  y = us_vs_funding['Fiscal Year_y'], c = "blue")
ax[2].set_title("Number of US Launches vs. NASA Funding", fontsize = '24')
ax[2].set_xlabel("NASA Funding")
ax[2].set_ylabel("Number of US Launches")


fig.savefig("funding_launch.png")



In [ ]:
# read in the dataset from the Union of Concerned Scientists
satellite_reasons = pd.read_csv('data/satellite_reasons.csv')

# merge new data to our original space object data
satellite_reasons_full = satellite_reasons.merge(all_objects, left_on = "NORAD Number", right_on = "NORAD_CAT_ID")

full_new = satellite_reasons_full.merge(jonathan, left_on = "NORAD Number", right_on = "Satcat")

full_new = full_new.loc[:, ~full_new.columns.str.startswith('Unnamed')]

# full_new.to_csv('data/full.csv')


satellite_reasons_full 

In [ ]:
# we only want the recent data where the satellite_reasons data has information about the object
recent = satellite_reasons_full[satellite_reasons_full['Class of Orbit']!= 'na']

year_range_recent = range(1974, 2020)

# count number of satellites in orbit in each orbit class each year
leo_count, geo_count, meo_count, elliptical_count = [], [], [], []

for year in year_range_recent:
    leo_count.append(np.sum((recent[recent['Class of Orbit'] == 'LEO']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Class of Orbit'] == 'LEO']["DECAY_YEAR"])))
    geo_count.append(np.sum((recent[recent['Class of Orbit'] == 'GEO']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Class of Orbit'] == 'GEO']["DECAY_YEAR"])))
    meo_count.append(np.sum((recent[recent['Class of Orbit'] == 'MEO']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Class of Orbit'] == 'MEO']["DECAY_YEAR"])))
    elliptical_count.append(np.sum((recent[recent['Class of Orbit'] == 'Elliptical']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Class of Orbit'] == 'Elliptical']["DECAY_YEAR"])))



In [ ]:
# count number of satellites in orbit for user/owner type during each year
commercial_count, civil_count, military_count, government_count, mixed_count = [], [], [], [], []

for year in year_range_recent:
    commercial_count.append(np.sum((recent[recent['Users'] == 'Commercial']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Users'] == 'Commercial']["DECAY_YEAR"])))
    civil_count.append(np.sum((recent[recent['Users'] == 'Civil']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Users'] == 'Civil']["DECAY_YEAR"])))
    military_count.append(np.sum((recent[recent['Users'] == 'Military']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Users'] == 'Military']["DECAY_YEAR"])))
    government_count.append(np.sum((recent[recent['Users'] == 'Government']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Users'] == 'Government']["DECAY_YEAR"])))



In [ ]:
# count number of satellites in orbit for each purpose type during each year
earth_obs_count, tech_dev_count, comms_count,space_science_count,nav_count = [], [], [], [], []

for year in year_range_recent:
    earth_obs_count.append(np.sum((recent[recent['Purpose'] == 'Earth Observation']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Purpose'] == 'Earth Observation']["DECAY_YEAR"])))
    tech_dev_count.append(np.sum((recent[recent['Purpose'] == 'Technology Development']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Purpose'] == 'Technology Development']["DECAY_YEAR"])))
    comms_count.append(np.sum((recent[recent['Purpose'] == 'Communications']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Purpose'] == 'Communications']["DECAY_YEAR"])))
    space_science_count.append(np.sum((recent[recent['Purpose'] == 'Space Science']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Purpose'] == 'Space Science']["DECAY_YEAR"])))
    nav_count.append(np.sum((recent[recent['Purpose'] == 'Navigation/Global Positioning']["LAUNCH_YEAR"] < year) & (year <= recent[recent['Purpose'] == 'Navigation/Global Positioning']["DECAY_YEAR"])))


In [ ]:
# plot number of satellites across various metrics

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(28,14))

ax[0].plot(year_range_recent, leo_count, c = "magenta", label = "LEO")
ax[0].plot(year_range_recent, geo_count, c = "purple", label = "GEO")
ax[0].plot(year_range_recent, meo_count, c = "deeppink", label = "MEO")
ax[0].plot(year_range_recent, elliptical_count, c = "y", label = "Elliptical")
ax[0].set_title("Satellites in Orbit by Type of Orbit", fontsize = '24')
ax[0].set_xlabel("Year")
ax[0].set_ylabel("Number of Satellites in Orbit")
ax[0].legend(fontsize = 'x-large')

ax[1].plot(year_range_recent, commercial_count, c = "red", label = "Commercial")
ax[1].plot(year_range_recent, civil_count, c = "blue", label = "Civil")
ax[1].plot(year_range_recent, military_count, c = "green", label = "Military")
ax[1].plot(year_range_recent, government_count, c = "orange", label = "Government" )
ax[1].set_title("Satellites in Orbit by Type of User", fontsize = '24')
ax[1].set_xlabel("Year")
ax[1].set_ylabel("Number of Satellites")
ax[1].legend(fontsize = 'x-large')


ax[2].plot(year_range_recent, earth_obs_count, c = "deepskyblue", label = "Earth Observation")
ax[2].plot(year_range_recent, tech_dev_count, c = "yellowgreen", label ="Technology Development")
ax[2].plot(year_range_recent, comms_count, c = "maroon", label = "Communications")
ax[2].plot(year_range_recent, space_science_count, c = "springgreen", label = "Space Science")
ax[2].plot(year_range_recent, nav_count, c = "navy", label = "Navigation/Global Positioning")
ax[2].set_title("Satellites in Orbit by Type of Purpose", fontsize = '24')
ax[2].set_xlabel("Year")
ax[2].set_ylabel("Number of Satellites")
ax[2].legend(fontsize = 'x-large')


#plt.savefig("satellites_by_cat.png")
